# Calculating catches from gridded DBPM outputs
**Author**: Denisse Fierro Arcos  
**Date**: 2025-08-06  

In this notebook, we will calculate biomass for detritivores and predators from gridded DBPM outputs.

## Loading relevant libraries

In [1]:
import os
os.chdir('/g/data/vf71/la6889/dbpm_southern_ocean/scripts/')
import useful_functions as uf
from glob import glob
import xarray as xr
import json
from dask.distributed import Client

## Start a cluster for parallelisation

In [3]:
client = Client(threads_per_worker = 1, connection_limit = 0)

## Defining basic variables to run gridded DBPM

In [13]:
#Name of region and model resolution
reg_number = 'fao-58'
reg_name = 'east_antarctica'
model_res = '1deg'

#Defining input and output folders
base_folder = f'/g/data/vf71/la6889/dbpm_inputs/{reg_name}'
gridded_inputs = os.path.join(base_folder, 'gridded_params', model_res)
gridded_outputs = os.path.join(base_folder, 'run_fishing', model_res)
outputs_folder = os.path.join(base_folder, 'gridded_dbpm_outputs', model_res)

#Ensure outputs folder exists
os.makedirs(outputs_folder, exist_ok = True)

## Loading gridded parameters and gridded inputs

In [15]:
#Gridded DBPM parameters
gridded_params = json.load(open(
    os.path.join(gridded_inputs, f'dbpm_gridded_size_params_{reg_number}_python.json')))

#Size class bins
log10_size_bins_mat = xr.open_zarr('outputs/log10_size_bins_matrix.zarr/')['size_bins']
size_bin_vals = (10**log10_size_bins_mat)

#Area - to be used in weighted mean calculation
area = xr.open_zarr(glob(os.path.join(base_folder, 'gridded', model_res, 
                                      '*areacello*'))[0])['cellareao']
area = area.fillna(0)

## Calculate biomass for predators and detritivores

In [66]:
#Predators
var_pred = 'predators'
pred_raw = (xr.open_mfdataset(glob(
    os.path.join(gridded_outputs, f'{var_pred}*')))[var_pred])

#Detritivores
var_det = 'detritivores'
det_raw = (xr.open_mfdataset(glob(
    os.path.join(gridded_outputs, f'{var_det}*')))[var_det])

In [67]:
pred = ((pred_raw*size_bin_vals*
        gridded_params['log_size_increase']).
    isel(size_class = slice(gridded_params['ind_min_pred_size'],
                            None)))
pred.name = 'pred_biomass'
pred['size_class'] = 10**pred.size_class.values
pred = pred.assign_attrs({'short_name': 'pred_biomass',
                          'long_name': 'Predator biomass per size class (in g)',
                          'units': 'g m-2'})

In [68]:
det = ((det_raw*size_bin_vals*
        gridded_params['log_size_increase']).
    isel(size_class = slice(gridded_params['ind_min_detritivore_size'],
                            None)))
det.name = 'detriti_biomass'
det['size_class'] = 10**det.size_class.values
det = det.assign_attrs({'short_name': 'detriti_biomass',
                        'long_name': 'Detritivore biomass per size class (in g)',
                        'units': 'g m-2'})

### Save outputs

In [69]:
pred.to_zarr(
    os.path.join(outputs_folder,
                 f'pred_biomass_size_group_{reg_number}_1841-2010.zarr'),
    consolidated = True, mode = 'w')

det.to_zarr(
    os.path.join(outputs_folder,
                 f'detriti_biomass_size_group_{reg_number}_1841-2010.zarr'),
    consolidated = True, mode = 'w')

### Load biomass if previously calculated

In [16]:
pred = xr.open_zarr(os.path.join(
    outputs_folder,
    f'pred_biomass_size_group_{reg_number}_1841-2010.zarr'))['pred_biomass']

det = xr.open_zarr(os.path.join(
    outputs_folder,
    f'detriti_biomass_size_group_{reg_number}_1841-2010.zarr'))['detriti_biomass']

### **Optional:** Subset biomass for predators and detritivores
If different from the minimum body size given in the `gridded_params` variable.

In [5]:
pred = pred.sel(size_class = slice(10, None))
det = det.sel(size_class = slice(10, None))

### Calculate mean group biomass per area per time step

In [6]:
# Predators
pred_tot = pred.sum('size_class')
pred_tot_w = pred_tot.weighted(area)
pred_tot_w_mean = pred_tot_w.mean(('lat', 'lon')).load()
pred_tot_w_mean.name = 'tot_pred_bio'

# Detritivores
det_tot = det.sum('size_class')
det_tot_w = det_tot.weighted(area)
det_tot_w_mean = det_tot_w.mean(('lat', 'lon')).load()
det_tot_w_mean.name = 'tot_det_bio'


KeyboardInterrupt



### Calculate mean group exploitable biomass per area per time step

In [72]:
#Predators
pred_exp = ((pred_raw*size_bin_vals*
             gridded_params['log_size_increase']).
    isel(size_class = slice(gridded_params['ind_min_fish_pred'],
                            None))).sum('size_class')
pred_exp_w = pred_exp.weighted(area)
pred_exp_w_mean = pred_exp_w.mean(('lat', 'lon')).load()
pred_exp_w_mean.name = 'tot_expl_pred_bio'

#Detritivores
det_exp = ((det_raw*size_bin_vals*
             gridded_params['log_size_increase']).
    isel(size_class = slice(gridded_params['ind_min_fish_det'],
                            None))).sum('size_class')
det_exp_w = det_exp.weighted(area)
det_exp_w_mean = det_exp_w.mean(('lat', 'lon')).load()
det_exp_w_mean.name = 'tot_expl_det_bio'

## Saving results as tables

In [73]:
bio_pred_det = (xr.merge([pred_exp_w_mean, pred_tot_w_mean, 
                          det_exp_w_mean, det_tot_w_mean]).to_dataframe().
    reset_index())
bio_pred_det['region'] = reg_number.upper().replace('-', ' ')
bio_pred_det['resolution'] = model_res

In [75]:
bio_pred_det.to_parquet(
    os.path.join(outputs_folder,
                 # f'mean_pred_det_bio_{reg_number}_1841-2010.parquet'),
                 f'mean_pred_det_10g-1t-bio_exploit-bio_{reg_number}_1841-2010.parquet'),
    index = False)